In [1]:
import datetime
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
from soykeyword.proportion import CorpusbasedKeywordExtractor
from collections import Counter

In [2]:
%%time
data = pd.read_csv('news_df_02100201.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161217 entries, 0 to 161216
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        161217 non-null  int64 
 1   Company   161217 non-null  int64 
 2   Year      161217 non-null  int64 
 3   Headline  161215 non-null  object
 4   Content   161217 non-null  object
 5   News      161217 non-null  object
 6   Subject   161217 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 8.6+ MB
CPU times: user 6.16 s, sys: 816 ms, total: 6.97 s
Wall time: 6.97 s


In [3]:
data.head()

,ID,Company,Year,Headline,Content,News,Subject
0,20191103192315001,2100201,2019,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복,머니투데이 방콕태국김성휘 기자 경쟁력 높은 시장 진출 위한 교두보 주형철 청와대 ...,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복머니투데이 방콕태국김성휘 기자 ...,0
1,20191103153210001,2100201,2019,사진코리아 세일 페스타 즐겨볼까,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중...,사진코리아 세일 페스타 즐겨볼까머니투데이 김휘선 기자 코리아세일페스타가 시작한 ...,0
2,20191103120110001,2100201,2019,대기업 거래 신평사 지정 관행 부담,머니투데이 김지훈 기자 중기중앙회 대기업 자발적 거래관행 개선 촉구 자료중소기업중앙...,대기업 거래 신평사 지정 관행 부담머니투데이 김지훈 기자 중기중앙회 대기업 자발...,0
3,20191103173816001,2100201,2019,년째 썩지않은 맥도날드 치즈버거 정말,머니투데이 김도엽 인턴 아이슬란드의 한 고객이 년 구매 현재도 썩지 않고 그대로 보...,년째 썩지않은 맥도날드 치즈버거 정말머니투데이 김도엽 인턴 아이슬란드의 한 고객이 ...,3
4,20191103152648001,2100201,2019,사진첫 주말 코리아 세일 페스타 북적이는 명동,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중구...,사진첫 주말 코리아 세일 페스타 북적이는 명동머니투데이 김휘선 기자 코리아세일페스...,0


In [4]:
ti = []
for i in data['ID']:
    ti.append(datetime.datetime.strptime(str(i)[:8], '%Y%m%d'))
data['Time'] = ti

In [5]:
num = int(len(data) * 0.7)
train_data = data[:num]
test_data = data[num:]

In [6]:
# 훈련, 테스트 분리
train_data = data[:110000]
test_data = data[110000:]
print(len(train_data))
print(len(test_data))

110000
51217


# 키워드 생성

soykeyword를 이용하여 기사 데이터 중 keyword로 인식할 만한 단어를 추출하여 저장합니다

In [7]:
mecab = Mecab() # 명사 추출 모듈 불러오기

# 키워드 추출기 생성
# min_tf : 단어 빈도, min_df : 문서 빈도
# min_tf = 100, min_df = 10
corpusbased_extractor = CorpusbasedKeywordExtractor(
    min_tf=1,
    min_df=1,
    tokenize=mecab.nouns,
    verbose=True
)

In [8]:
%%time
corpusbased_extractor.train(train_data['News'])

training was done 55160 terms, 110000 docs, memory = 2.918 Gb82 Gb
CPU times: user 2min 21s, sys: 1.14 s, total: 2min 22s
Wall time: 2min 23s


In [9]:
len(corpusbased_extractor._d2t[10])

46

In [10]:
len(mecab.nouns(train_data['News'][10]))

81

In [11]:
# corpusbased_extractor._tfs.keys() # 키워드 추출 목록
# corpusbased_extractor._t2d

In [12]:
cnt = 0 
for i in corpusbased_extractor._tfs.keys():
    if len(i) == 1:
        cnt += 1
cnt # 길이가 1인 키워드 갯수

1069

In [13]:
%%time
word = [] # 함수에 쓰일 단어장
for i in corpusbased_extractor._tfs.keys():
    if len(i) > 1:
        word.append(i)
len(word)

CPU times: user 15.7 ms, sys: 793 µs, total: 16.5 ms
Wall time: 16.1 ms


54091

In [14]:
cnt = 0 
for i in word:
    if len(i) == 1:
        cnt += 1
cnt # 길이가 1인 키워드 갯수

0

In [15]:
%%time
with open('keyword_mecab.txt', 'w') as f: # 함수에 쓰일 단어장 저장
    for line in word:
        f.write(line + ' ')

CPU times: user 17.6 ms, sys: 1.68 ms, total: 19.3 ms
Wall time: 18.9 ms


In [16]:
%%time
with open('keyword_mecab.txt', 'r') as t: # 잘 저장 되었나 확인
    word = t.readlines()
word = word[0].split()
len(word)

CPU times: user 7.09 ms, sys: 2.53 ms, total: 9.63 ms
Wall time: 8.5 ms


54091

In [17]:
# corpusbased_extractor._d2t[1300] # 문서당 키워드와 언급된 횟수

In [18]:
%%time
kw_data = [] # 문서당 키워드
for i in range(len(corpusbased_extractor._d2t)):
    rd_data = []
    for j, k in enumerate(corpusbased_extractor._d2t[i]):
        if len(k[0]) > 1:
            rd_data.append(k[0])
        if len(rd_data) == 10: # 키워드 많이 언급된 상위 10개 추출
            break
    kw_data.append(rd_data)
train_data['Keyword'] = kw_data

CPU times: user 2.26 s, sys: 17.7 ms, total: 2.28 s
Wall time: 2.27 s


<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
train_data.head()

,ID,Company,Year,Headline,Content,News,Subject,Time,Keyword
0,20191103192315001,2100201,2019,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복,머니투데이 방콕태국김성휘 기자 경쟁력 높은 시장 진출 위한 교두보 주형철 청와대 ...,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복머니투데이 방콕태국김성휘 기자 ...,0,2019-11-03,"[아세안, 인도네시아, 정상, 회의, 타결, 태국, 양자, 보좌관, 특별, 일본]"
1,20191103153210001,2100201,2019,사진코리아 세일 페스타 즐겨볼까,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중...,사진코리아 세일 페스타 즐겨볼까머니투데이 김휘선 기자 코리아세일페스타가 시작한 ...,0,2019-11-03,"[페스타, 세일, 코리아, 명동, 관광객, 주말, 휘선, 거리, 중구, 시민]"
2,20191103120110001,2100201,2019,대기업 거래 신평사 지정 관행 부담,머니투데이 김지훈 기자 중기중앙회 대기업 자발적 거래관행 개선 촉구 자료중소기업중앙...,대기업 거래 신평사 지정 관행 부담머니투데이 김지훈 기자 중기중앙회 대기업 자발...,0,2019-11-03,"[신용, 중소기업, 거래, 평가서, 기업, 발급, 평가, 관행, 요구, 중앙]"
3,20191103173816001,2100201,2019,년째 썩지않은 맥도날드 치즈버거 정말,머니투데이 김도엽 인턴 아이슬란드의 한 고객이 년 구매 현재도 썩지 않고 그대로 보...,년째 썩지않은 맥도날드 치즈버거 정말머니투데이 김도엽 인턴 아이슬란드의 한 고객이 ...,3,2019-11-03,"[햄버거, 스노, 감자튀김, 맥도날드, 아이슬란드, 하우스, 트라, 구매, 버거, 보관]"
4,20191103152648001,2100201,2019,사진첫 주말 코리아 세일 페스타 북적이는 명동,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중구...,사진첫 주말 코리아 세일 페스타 북적이는 명동머니투데이 김휘선 기자 코리아세일페스...,0,2019-11-03,"[페스타, 세일, 명동, 주말, 코리아, 관광객, 휘선, 거리, 중구, 시민]"


In [20]:
train_data.to_csv('news_recommend.csv', index = False)

In [21]:
train_data = pd.read_csv('news_recommend.csv')
train_data.head()

,ID,Company,Year,Headline,Content,News,Subject,Time,Keyword
0,20191103192315001,2100201,2019,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복,머니투데이 방콕태국김성휘 기자 경쟁력 높은 시장 진출 위한 교두보 주형철 청와대 ...,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복머니투데이 방콕태국김성휘 기자 ...,0,2019-11-03,"['아세안', '인도네시아', '정상', '회의', '타결', '태국', '양자',..."
1,20191103153210001,2100201,2019,사진코리아 세일 페스타 즐겨볼까,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중...,사진코리아 세일 페스타 즐겨볼까머니투데이 김휘선 기자 코리아세일페스타가 시작한 ...,0,2019-11-03,"['페스타', '세일', '코리아', '명동', '관광객', '주말', '휘선', ..."
2,20191103120110001,2100201,2019,대기업 거래 신평사 지정 관행 부담,머니투데이 김지훈 기자 중기중앙회 대기업 자발적 거래관행 개선 촉구 자료중소기업중앙...,대기업 거래 신평사 지정 관행 부담머니투데이 김지훈 기자 중기중앙회 대기업 자발...,0,2019-11-03,"['신용', '중소기업', '거래', '평가서', '기업', '발급', '평가', ..."
3,20191103173816001,2100201,2019,년째 썩지않은 맥도날드 치즈버거 정말,머니투데이 김도엽 인턴 아이슬란드의 한 고객이 년 구매 현재도 썩지 않고 그대로 보...,년째 썩지않은 맥도날드 치즈버거 정말머니투데이 김도엽 인턴 아이슬란드의 한 고객이 ...,3,2019-11-03,"['햄버거', '스노', '감자튀김', '맥도날드', '아이슬란드', '하우스', ..."
4,20191103152648001,2100201,2019,사진첫 주말 코리아 세일 페스타 북적이는 명동,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중구...,사진첫 주말 코리아 세일 페스타 북적이는 명동머니투데이 김휘선 기자 코리아세일페스...,0,2019-11-03,"['페스타', '세일', '명동', '주말', '코리아', '관광객', '휘선', ..."
